In [1]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.4/423.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3

In [2]:
# -*- coding: utf-8 -*-
# ... (라이브러리 임포트 - 이전과 동일) ...
import os, glob, json, hashlib, torch
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
import pandas as pd

# --- 1. 설정 (Configuration) ---
# ... (INPUT_DIR, OUTPUT_DIR, MODEL_NAME, BATCH_SIZE 등 설정 - 이전과 동일) ...
INPUT_DIR = '/kaggle/input/predata/'
OUTPUT_DIR = '/kaggle/working/output'
OUTPUT_EMBEDDING_FILE = os.path.join(OUTPUT_DIR, 'chunk_embeddings.jsonl')
MODEL_NAME = 'jhgan/ko-sbert-sts'
BATCH_SIZE = 32
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"입력 디렉토리: {INPUT_DIR}")
print(f"출력 디렉토리: {OUTPUT_DIR}")
print(f"출력 파일: {OUTPUT_EMBEDDING_FILE}")

# --- 2. 모델 및 환경 설정 ---
# ... (GPU 설정, 모델 로딩, Splitter 생성 - 이전과 동일, 오류 처리 포함) ...
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")
embedding_model = None
text_splitter = None
try:
    print(f"'{MODEL_NAME}' 임베딩 모델 로딩 시작...")
    embedding_model = SentenceTransformer(MODEL_NAME, device=device)
    print("임베딩 모델 로딩 완료.")
    langchain_embeddings = HuggingFaceEmbeddings(
        model_name=MODEL_NAME, model_kwargs={'device': device}, encode_kwargs={'normalize_embeddings': False}
    )
    print("LangChain Embeddings 래퍼 생성 완료.")
    text_splitter = SemanticChunker(langchain_embeddings, breakpoint_threshold_type="percentile")
    print("SemanticChunker 생성 완료.")
except Exception as e:
    print(f"모델 또는 Splitter 초기화 중 오류 발생: {e}")

# --- 3. 이어하기 기능: 기존 데이터 로드 (ID와 해시 포함) --- ### 수정됨 ###

existing_data_dict = {} # 기존 데이터를 저장할 딕셔너리 {chunk_unique_id: data_dict}
if os.path.exists(OUTPUT_EMBEDDING_FILE):
    print(f"기존 출력 파일 '{OUTPUT_EMBEDDING_FILE}'을 읽어옵니다...")
    try:
        with open(OUTPUT_EMBEDDING_FILE, 'r', encoding='utf-8') as f_in:
            for line in f_in:
                try:
                    data = json.loads(line)
                    note_id = data.get('note_id')
                    chunk_id = data.get('chunk_id')
                    # text_hash와 vector가 있는지 확인 (이전 버전 파일 호환성)
                    text_hash = data.get('text_hash')
                    vector = data.get('vector')
                    if note_id is not None and chunk_id is not None and text_hash and vector:
                        chunk_unique_id = f"{note_id}_{chunk_id}"
                        existing_data_dict[chunk_unique_id] = data # 전체 데이터 저장
                    # else:
                    #     print(f"경고: 기존 데이터에 필요한 필드(text_hash, vector 등)가 누락되었습니다. 해당 라인 건너뛰기: {line.strip()}")
                except json.JSONDecodeError:
                    print(f"경고: 출력 파일의 잘못된 JSON 라인 건너뛰기: {line.strip()}")
        print(f"총 {len(existing_data_dict)}개의 기존 청크 데이터를 로드했습니다.")
    except Exception as e:
        print(f"경고: 기존 출력 파일을 읽는 중 오류 발생: {e}. 처음부터 다시 처리합니다.")
        existing_data_dict = {} # 오류 시 초기화

# --- 4. 데이터 처리 및 임베딩 대상 선정 --- ### 수정됨 ###

if embedding_model and text_splitter:
    print("\n--- 청크 분할 및 변경 사항 확인 시작 ---")

    chunks_to_embed = [] # 새로 임베딩해야 할 청크 데이터 리스트
    processed_this_run = {} # 현재 실행에서 유효한 데이터 저장 (기존+신규+업데이트)

    input_files = glob.glob(os.path.join(INPUT_DIR, '*.json'))
    print(f"총 {len(input_files)}개의 JSON 파일을 찾았습니다.")

    for filepath in tqdm(input_files, desc="파일 읽기 및 청킹/비교 중"):
        display_filepath = os.path.basename(filepath)
        try:
            with open(filepath, 'r', encoding='utf-8-sig') as f_in: content = json.load(f_in)

            if 'data' in content and isinstance(content['data'], list):
                for doc_data in content['data']:
                    note_id = doc_data.get('source_id'); corpus = doc_data.get('corpus')
                    doc_title = doc_data.get('title', '')
                    if not note_id or not corpus or not isinstance(corpus, str): continue

                    current_chunk_id_counter = 0
                    try:
                        chunks = text_splitter.split_text(corpus)
                    except Exception as split_err:
                        print(f"오류: {display_filepath} (ID: {note_id}) 청킹 중 오류: {split_err}")
                        continue

                    for chunk_text in chunks:
                        if not chunk_text.strip(): continue
                        chunk_id = current_chunk_id_counter
                        chunk_unique_id = f"{note_id}_{chunk_id}"
                        current_hash = hashlib.sha256(chunk_text.strip().encode('utf-8')).hexdigest()

                        # *** 이어하기 + 변경 감지 로직 ***
                        process_this_chunk = False
                        previous_data = existing_data_dict.get(chunk_unique_id)

                        if previous_data is None: # 신규 청크
                            process_this_chunk = True
                        elif previous_data.get('text_hash') != current_hash: # 내용 변경됨
                            # print(f"정보: 내용 변경 감지, 재처리: {chunk_unique_id}") # 로그 (선택적)
                            process_this_chunk = True

                        # 현재 실행에서 유효한 데이터 구성
                        current_chunk_data = {
                            "note_id": note_id, "chunk_id": chunk_id,
                            "doc_title": doc_title, "filepath": filepath,
                            "text": chunk_text.strip(), "text_hash": current_hash
                            # 'vector'는 나중에 추가됨
                        }

                        if process_this_chunk:
                            chunks_to_embed.append(current_chunk_data) # 임베딩 대상 리스트에 추가
                        else:
                            # 변경 없는 경우, 기존 벡터 사용
                            current_chunk_data['vector'] = previous_data.get('vector')

                        # 현재 실행의 최종 데이터 딕셔너리에 저장/업데이트
                        processed_this_run[chunk_unique_id] = current_chunk_data
                        current_chunk_id_counter += 1

            else: print(f"경고: 파일 {display_filepath} 건너뛰기 ('data' 리스트 없음).")
        except json.JSONDecodeError as json_err: print(f"오류: 파일 {display_filepath} JSON 구조 오류: {json_err}")
        except Exception as file_err: print(f"파일 {display_filepath} 처리 중 오류: {file_err}")

    # --- 5. 배치 임베딩 (새 대상만) ---
    newly_processed_count = len(chunks_to_embed)
    print(f"\n총 {newly_processed_count}개의 신규/변경된 청크에 대해 임베딩을 시작합니다.")

    if newly_processed_count > 0:
        for i in tqdm(range(0, newly_processed_count, BATCH_SIZE), desc="임베딩 중"):
            batch_data_to_embed = chunks_to_embed[i : i + BATCH_SIZE]
            batch_texts = [item['text'] for item in batch_data_to_embed]
            if not batch_texts: continue

            try:
                batch_embeddings = embedding_model.encode(
                    batch_texts, convert_to_numpy=True, show_progress_bar=False, batch_size=len(batch_texts)
                )
                batch_embeddings_list = batch_embeddings.tolist()

                # 계산된 벡터를 processed_this_run 딕셔너리에 업데이트
                for j, embedding_vector in enumerate(batch_embeddings_list):
                    # 임베딩 대상 리스트의 원본 딕셔너리를 직접 수정하면 안됨
                    # processed_this_run 에서 해당 청크를 찾아 업데이트
                    original_chunk_data = batch_data_to_embed[j]
                    chunk_unique_id = f"{original_chunk_data['note_id']}_{original_chunk_data['chunk_id']}"
                    if chunk_unique_id in processed_this_run:
                         processed_this_run[chunk_unique_id]['vector'] = embedding_vector
                    # else: # 이론상 이 경우는 없어야 함
                    #     print(f"경고: 임베딩된 청크 {chunk_unique_id}를 processed_this_run에서 찾을 수 없습니다.")

            except Exception as embed_err:
                print(f"오류: 배치 {i // BATCH_SIZE} 임베딩 중 오류: {embed_err}")
    else:
        print("새로 처리할 청크가 없습니다.")

    # --- 6. 최종 결과 파일 저장 (전체 덮어쓰기) --- ### 수정됨 ###
    print(f"\n--- 최종 결과 파일 저장 시작 ({len(processed_this_run)}개 청크) ---")
    # 출력 파일을 쓰기 모드('w')로 열어 전체 내용을 새로 씀
    try:
        with open(OUTPUT_EMBEDDING_FILE, 'w', encoding='utf-8') as f_out:
            # 정렬 기준 설정 (note_id, chunk_id 순서) - 선택 사항이지만 일관성에 좋음
            sorted_chunk_ids = sorted(processed_this_run.keys(), key=lambda x: (x.split('_')[0], int(x.split('_')[1])))

            for chunk_unique_id in tqdm(sorted_chunk_ids, desc="최종 파일 저장 중"):
                final_chunk_data = processed_this_run[chunk_unique_id]
                # 벡터 데이터가 있는지 최종 확인 (임베딩 오류 등으로 누락될 수 있음)
                if 'vector' in final_chunk_data and final_chunk_data['vector']:
                    json_string = json.dumps(final_chunk_data, ensure_ascii=False)
                    f_out.write(json_string + '\n')
                else:
                    print(f"경고: 청크 {chunk_unique_id}의 벡터 데이터가 없어 최종 파일에 저장하지 않습니다.")
        print(f"최종 결과가 '{OUTPUT_EMBEDDING_FILE}'에 저장되었습니다.")
    except Exception as write_err:
        print(f"오류: 최종 결과 파일 '{OUTPUT_EMBEDDING_FILE}' 저장 중 오류 발생: {write_err}")


    print("\n--- 임베딩 생성 및 저장 완료 ---")
    print("-" * 40)

else:
    print("\n오류: 모델 또는 텍스트 스플리터가 제대로 초기화되지 않아 임베딩 프로세스를 시작할 수 없습니다.")

입력 디렉토리: /kaggle/input/predata/
출력 디렉토리: /kaggle/working/output
출력 파일: /kaggle/working/output/chunk_embeddings.jsonl
사용 장치: cuda
'jhgan/ko-sbert-sts' 임베딩 모델 로딩 시작...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

LangChain Embeddings 래퍼 생성 완료.
SemanticChunker 생성 완료.
기존 출력 파일 '/kaggle/working/output/chunk_embeddings.jsonl'을 읽어옵니다...
총 130개의 기존 청크 데이터를 로드했습니다.

--- 청크 분할 및 변경 사항 확인 시작 ---
총 10개의 JSON 파일을 찾았습니다.


파일 읽기 및 청킹/비교 중:   0%|          | 0/10 [00:00<?, ?it/s]


총 0개의 신규/변경된 청크에 대해 임베딩을 시작합니다.
새로 처리할 청크가 없습니다.

--- 최종 결과 파일 저장 시작 (130개 청크) ---


최종 파일 저장 중:   0%|          | 0/130 [00:00<?, ?it/s]

최종 결과가 '/kaggle/working/output/chunk_embeddings.jsonl'에 저장되었습니다.

--- 임베딩 생성 및 저장 완료 ---
----------------------------------------


In [3]:
# -*- coding: utf-8 -*-
# 필요한 라이브러리 임포트
import os
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import networkx as nx # 그래프 기반 그룹핑을 위해 사용 (설치 필요 시 !pip install networkx)
from tqdm.notebook import tqdm

# --- 1. 설정 (Configuration) ---

INPUT_EMBEDDING_FILE = '/kaggle/working/output/chunk_embeddings.jsonl' # 임베딩 데이터 파일
OUTPUT_GROUP_FILE = '/kaggle/working/output/grouped_chunks_info.jsonl' # 그룹핑 결과 파일
SIMILARITY_THRESHOLD = 0.7  # 유사도 임계값 (실험적으로 조절 필요)
MIN_GROUP_SIZE = 2          # 그룹으로 간주할 최소 청크 수 (예: 2개 이상 유사해야 그룹)

print(f"입력 임베딩 파일: {INPUT_EMBEDDING_FILE}")
print(f"출력 그룹 파일: {OUTPUT_GROUP_FILE}")
print(f"유사도 임계값: {SIMILARITY_THRESHOLD}")
print(f"최소 그룹 크기: {MIN_GROUP_SIZE}")

# --- 2. 데이터 로드 ---

all_chunk_data = [] # 청크 데이터 (메타데이터 + 벡터) 저장 리스트
chunk_embeddings = [] # 벡터만 따로 저장할 리스트 (유사도 계산용)
chunk_lookup = {}     # 인덱스 -> 청크 메타데이터 매핑용 딕셔너리

print(f"\n--- 임베딩 데이터 로딩 시작 ---")
if not os.path.exists(INPUT_EMBEDDING_FILE):
    print(f"오류: 임베딩 파일 '{INPUT_EMBEDDING_FILE}'을 찾을 수 없습니다.")
else:
    try:
        with open(INPUT_EMBEDDING_FILE, 'r', encoding='utf-8') as f_in:
            for i, line in enumerate(f_in):
                try:
                    data = json.loads(line)
                    # 필수 데이터 확인
                    if 'note_id' in data and 'chunk_id' in data and 'vector' in data and 'text' in data:
                        vector = np.array(data['vector'], dtype=np.float32) # numpy 배열로 변환
                        # 벡터 차원 일관성 확인 (선택적)
                        if i > 0 and vector.shape[0] != chunk_embeddings[0].shape[0]:
                             print(f"경고: {i}번째 청크 벡터 차원({vector.shape[0]})이 이전 벡터 차원({chunk_embeddings[0].shape[0]})과 다릅니다. 건너<0xEB><0x9B><0x84>니다.")
                             continue

                        all_chunk_data.append(data) # 전체 데이터 저장
                        chunk_embeddings.append(vector) # 벡터만 따로 저장
                        chunk_lookup[i] = data # 인덱스로 메타데이터 조회 가능하도록
                    else:
                        print(f"경고: {i}번째 라인에 필수 필드가 누락되어 건너<0xEB><0x9B><0x84>니다.")
                except json.JSONDecodeError:
                    print(f"경고: 잘못된 JSON 라인 건너뛰기: {line.strip()}")
                except Exception as parse_err:
                     print(f"경고: 데이터 처리 중 오류 발생 ({i}번째 라인): {parse_err}")

        if not all_chunk_data or not chunk_embeddings:
             print("오류: 유효한 임베딩 데이터를 로드하지 못했습니다.")
             chunk_embeddings = None # 이후 처리 방지
        else:
            chunk_embeddings = np.array(chunk_embeddings) # 최종적으로 numpy 배열로 변환
            print(f"총 {len(all_chunk_data)}개의 청크 데이터 및 임베딩 로드 완료.")
            print(f"임베딩 배열 형태: {chunk_embeddings.shape}")

    except Exception as e:
        print(f"임베딩 파일 로딩 중 오류 발생: {e}")
        chunk_embeddings = None

# --- 3. 유사도 계산 및 그룹핑 ---

if chunk_embeddings is not None and len(chunk_embeddings) >= MIN_GROUP_SIZE:
    print(f"\n--- 유사도 계산 및 그룹핑 시작 (임계값: {SIMILARITY_THRESHOLD}) ---")

    # 코사인 유사도 계산 (모든 쌍)
    print("코사인 유사도 행렬 계산 중...")
    # 벡터 정규화 (L2 norm) - cosine 유사도는 내적(dot product)으로 계산 가능
    # SentenceTransformer 모델이 이미 정규화된 벡터를 반환할 수도 있지만, 안전하게 다시 정규화
    # from sklearn.preprocessing import normalize
    # embeddings_normalized = normalize(chunk_embeddings, axis=1, norm='l2')
    # similarity_matrix = np.dot(embeddings_normalized, embeddings_normalized.T)
    # 또는 sklearn 함수 직접 사용 (내부적으로 정규화 처리 가능성 있음)
    similarity_matrix = cosine_similarity(chunk_embeddings)
    print("유사도 행렬 계산 완료.")

    # 임계값 이상인 쌍들을 엣지로 하는 그래프 생성
    print("유사도 기반 그래프 생성 중...")
    graph = nx.Graph()
    num_chunks = len(all_chunk_data)
    # 자기 자신과의 유사도는 제외하고, 임계값 이상인 엣지만 추가
    for i in tqdm(range(num_chunks), desc="엣지 추가 중"):
        graph.add_node(i) # 모든 청크를 노드로 추가
        for j in range(i + 1, num_chunks): # 중복 계산 피하기 위해 i+1 부터 시작
            if similarity_matrix[i, j] >= SIMILARITY_THRESHOLD:
                graph.add_edge(i, j, weight=similarity_matrix[i, j]) # 엣지 추가 (weight는 유사도)

    # 연결된 컴포넌트(그룹) 찾기
    print("연결된 컴포넌트(그룹) 찾는 중...")
    connected_components = list(nx.connected_components(graph))

    # 최소 크기 기준을 만족하는 그룹만 필터링
    valid_groups = [group for group in connected_components if len(group) >= MIN_GROUP_SIZE]
    print(f"총 {len(valid_groups)}개의 유효한 그룹 (크기 >= {MIN_GROUP_SIZE})을 찾았습니다.")

    # --- 4. 그룹 정보 및 액션 대상 생성/저장 ---
    print(f"\n--- 그룹 정보 및 액션 대상 파일 저장 시작 ---")
    grouped_results = []
    group_id_counter = 0

    for group_indices in tqdm(valid_groups, desc="그룹 정보 생성 중"):
        group_data = {
            "group_id": group_id_counter,
            "similarity_threshold": SIMILARITY_THRESHOLD,
            "member_chunks": [],
            "synthesis_input_texts": [],
            "backlink_candidate_notes": set() # 중복 제거 위해 Set 사용
        }

        # 그룹 멤버 정보 추가
        member_texts = []
        note_ids_in_group = set()
        for index in group_indices:
            chunk_info = chunk_lookup.get(index)
            if chunk_info:
                # 그룹 멤버 정보 구성 (필요한 메타데이터 추가)
                member_chunk_info = {
                    "note_id": chunk_info.get("note_id"),
                    "chunk_id": chunk_info.get("chunk_id"),
                    "doc_title": chunk_info.get("doc_title", ""),
                    "filepath": chunk_info.get("filepath", ""),
                    "text_preview": chunk_info.get("text", "")[:100] + "..." # 텍스트 미리보기
                    # 필요시 그룹 내 다른 멤버와의 평균/최대 유사도 등 추가 가능
                }
                group_data["member_chunks"].append(member_chunk_info)
                member_texts.append(chunk_info.get("text", "")) # 통합용 텍스트
                note_ids_in_group.add(chunk_info.get("note_id")) # 백링크용 노트 ID

        group_data["synthesis_input_texts"] = member_texts
        # Set을 리스트로 변환하여 저장
        group_data["backlink_candidate_notes"] = sorted(list(note_ids_in_group))

        grouped_results.append(group_data)
        group_id_counter += 1

    # 최종 결과를 JSON Lines 파일로 저장
    try:
        with open(OUTPUT_GROUP_FILE, 'w', encoding='utf-8') as f_out:
            for group_result in grouped_results:
                json_string = json.dumps(group_result, ensure_ascii=False)
                f_out.write(json_string + '\n')
        print(f"그룹핑 결과가 '{OUTPUT_GROUP_FILE}'에 저장되었습니다.")
    except Exception as write_err:
        print(f"오류: 그룹핑 결과 파일 저장 중 오류 발생: {write_err}")

    print("\n--- 유사 청크 그룹핑 및 액션 정보 생성 완료 ---")
    print("-" * 40)

elif len(chunk_embeddings) < MIN_GROUP_SIZE:
     print(f"\n오류: 로드된 청크 수가 최소 그룹 크기({MIN_GROUP_SIZE})보다 작아 그룹핑을 수행할 수 없습니다.")
else:
    print("\n오류: 임베딩 데이터가 로드되지 않아 그룹핑 프로세스를 시작할 수 없습니다.")

입력 임베딩 파일: /kaggle/working/output/chunk_embeddings.jsonl
출력 그룹 파일: /kaggle/working/output/grouped_chunks_info.jsonl
유사도 임계값: 0.7
최소 그룹 크기: 2

--- 임베딩 데이터 로딩 시작 ---
총 130개의 청크 데이터 및 임베딩 로드 완료.
임베딩 배열 형태: (130, 768)

--- 유사도 계산 및 그룹핑 시작 (임계값: 0.7) ---
코사인 유사도 행렬 계산 중...
유사도 행렬 계산 완료.
유사도 기반 그래프 생성 중...


엣지 추가 중:   0%|          | 0/130 [00:00<?, ?it/s]

연결된 컴포넌트(그룹) 찾는 중...
총 15개의 유효한 그룹 (크기 >= 2)을 찾았습니다.

--- 그룹 정보 및 액션 대상 파일 저장 시작 ---


그룹 정보 생성 중:   0%|          | 0/15 [00:00<?, ?it/s]

그룹핑 결과가 '/kaggle/working/output/grouped_chunks_info.jsonl'에 저장되었습니다.

--- 유사 청크 그룹핑 및 액션 정보 생성 완료 ---
----------------------------------------


In [4]:
# -*- coding: utf-8 -*-
import json
import os

# 그룹핑 결과 파일 경로
GROUP_INFO_FILE = '/kaggle/working/output/grouped_chunks_info.jsonl'

print(f"--- 그룹핑 결과 확인 ({GROUP_INFO_FILE}) ---")

if not os.path.exists(GROUP_INFO_FILE):
    print(f"오류: 그룹핑 결과 파일 '{GROUP_INFO_FILE}'을 찾을 수 없습니다.")
else:
    try:
        with open(GROUP_INFO_FILE, 'r', encoding='utf-8') as f_in:
            group_found = False
            for line in f_in:
                group_found = True
                try:
                    group_data = json.loads(line)
                    group_id = group_data.get("group_id", "N/A")
                    threshold = group_data.get("similarity_threshold", "N/A")
                    member_chunks = group_data.get("member_chunks", [])
                    backlink_notes = group_data.get("backlink_candidate_notes", [])

                    print(f"\n===== 그룹 ID: {group_id} (임계값: {threshold}) =====")
                    print(f"포함된 청크 수: {len(member_chunks)}")
                    print(f"백링크 후보 노트 ID: {backlink_notes}")

                    print("\n--- 포함된 청크 목록 (미리보기) ---")
                    if not member_chunks:
                        print("  (포함된 청크 정보 없음)")
                    else:
                        for i, chunk_info in enumerate(member_chunks):
                            note_id = chunk_info.get("note_id", "?")
                            chunk_id = chunk_info.get("chunk_id", "?")
                            title = chunk_info.get("doc_title", "제목 없음")
                            preview = chunk_info.get("text_preview", "내용 없음")
                            print(f"  {i+1}. [노트:{note_id} / 청크:{chunk_id}] (제목: {title})")
                            print(f"     내용: {preview}")

                    # 통합용 텍스트는 너무 길 수 있으니 필요한 경우 별도 확인
                    # synthesis_texts = group_data.get("synthesis_input_texts", [])
                    # print("\n--- 통합 대상 텍스트 목록 ---")
                    # for i, text in enumerate(synthesis_texts):
                    #     print(f"  {i+1}. {text[:150]}...") # 일부만 출력

                    print("=" * (len(str(group_id)) + 20)) # 구분선

                except json.JSONDecodeError:
                    print(f"\n오류: 잘못된 JSON 라인 발견 - {line.strip()}")
                except Exception as parse_err:
                    print(f"\n오류: 그룹 데이터 처리 중 오류 발생 - {parse_err}")

            if not group_found:
                print("결과 파일은 존재하지만, 유효한 그룹 정보를 찾지 못했습니다.")

    except Exception as e:
        print(f"그룹핑 결과 파일 읽기 중 오류 발생: {e}")

print("\n--- 그룹핑 결과 확인 완료 ---")

--- 그룹핑 결과 확인 (/kaggle/working/output/grouped_chunks_info.jsonl) ---

===== 그룹 ID: 0 (임계값: 0.7) =====
포함된 청크 수: 2
백링크 후보 노트 ID: ['S0000105']

--- 포함된 청크 목록 (미리보기) ---
  1. [노트:S0000105 / 청크:1] (제목: 횡성군 민원행정 고객만족도 설문조사 용역보고서 2006.하반기)
     내용: 연구배경 및 목적
1. 연구의 배경민원행정서비스에 대한 만족도 조사는 한국행정연구원에서 1996년 조사모델과 방법을 개발한 이후 현재까지 지속적으로 수행하고 있는 중앙정부 및 지방...
  2. [노트:S0000105 / 청크:3] (제목: 횡성군 민원행정 고객만족도 설문조사 용역보고서 2006.하반기)
     내용: 11. 30(35일)
▷ 공간적 범위
- 횡성군 종합민원실
▷ 내용적 범위
- 서론(연구 개요로서 연구의 배경과 목적 및 범위 및 방법)
- 민원 만족도 조사(민원서비스 시설, 민...

===== 그룹 ID: 1 (임계값: 0.7) =====
포함된 청크 수: 3
백링크 후보 노트 ID: ['S0000105']

--- 포함된 청크 목록 (미리보기) ---
  1. [노트:S0000105 / 청크:8] (제목: 횡성군 민원행정 고객만족도 설문조사 용역보고서 2006.하반기)
     내용: 6. 분산분석(ANOVA; analysis of variance)결과
가. 연령별 분산분석
1)연령별 민원시설관련 분산분석응답자들의 연령대별 평균차이를 비교하여 통계적으로 유의한 ...
  2. [노트:S0000105 / 청크:9] (제목: 횡성군 민원행정 고객만족도 설문조사 용역보고서 2006.하반기)
     내용: 나. 민원분야에 따른 분산분석
1)민원시설관련 만족도 분산분석응답자들이 제공받은 민원서비스 분야별 접근성에 대한 만족도는 평균 72.70점으로, 주차장과 관련한 만족도는 61.85...
  3. [노트:S00001

In [1]:
# -*- coding: utf-8 -*-
# 필요한 라이브러리 임포트
import os
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pandas as pd
from tqdm.notebook import tqdm

# --- 1. 설정 (Configuration) ---

INPUT_EMBEDDING_FILE = '/kaggle/working/output/chunk_embeddings.jsonl' # 임베딩 데이터 파일

print(f"입력 임베딩 파일: {INPUT_EMBEDDING_FILE}")

# --- 2. 데이터 로드 및 노트별 그룹핑 ---

notes_data = defaultdict(list) # {note_id: [vector1, vector2, ...]} 형태
note_chunk_counts = defaultdict(int) # 노트별 청크 수 저장

print(f"\n--- 임베딩 데이터 로드 및 노트별 그룹핑 시작 ---")
if not os.path.exists(INPUT_EMBEDDING_FILE):
    print(f"오류: 임베딩 파일 '{INPUT_EMBEDDING_FILE}'을 찾을 수 없습니다.")
else:
    try:
        line_count = 0
        with open(INPUT_EMBEDDING_FILE, 'r', encoding='utf-8') as f_in:
            for line in f_in:
                line_count += 1
                try:
                    data = json.loads(line)
                    note_id = data.get('note_id')
                    vector = data.get('vector')
                    if note_id and vector:
                        # 벡터를 float32 numpy 배열로 변환하여 추가
                        notes_data[note_id].append(np.array(vector, dtype=np.float32))
                        note_chunk_counts[note_id] += 1
                except json.JSONDecodeError:
                    print(f"경고: 잘못된 JSON 라인 건너뛰기 ({line_count}번째 라인)")
                except Exception as parse_err:
                     print(f"경고: 데이터 처리 중 오류 발생 ({line_count}번째 라인): {parse_err}")

        print(f"총 {len(notes_data)}개의 노트에서 {sum(note_chunk_counts.values())}개의 청크 임베딩 로드 완료.")

    except Exception as e:
        print(f"임베딩 파일 로딩 중 오류 발생: {e}")
        notes_data = None # 이후 처리 방지

# --- 3. 문서 내 유사도 계산 및 통계 분석 ---

if notes_data:
    print("\n--- 각 문서 내 청크 간 유사도 분포 분석 시작 ---")

    results_per_note = [] # 각 노트별 통계 저장 리스트
    all_similarities = [] # 전체 유사도 값 저장 리스트

    # 각 노트를 순회하며 분석
    for note_id, vectors in tqdm(notes_data.items(), desc="문서별 유사도 분석 중"):
        num_chunks = len(vectors)

        # 청크가 2개 이상인 경우에만 유사도 계산 가능
        if num_chunks >= 2:
            # 벡터 리스트를 numpy 배열로 변환
            embeddings_array = np.array(vectors)

            # 코사인 유사도 계산
            try:
                # 벡터가 이미 정규화되었다고 가정하고 내적 사용 or cosine_similarity 직접 사용
                # sim_matrix = np.dot(embeddings_array, embeddings_array.T)
                sim_matrix = cosine_similarity(embeddings_array)

                # 대각선(자기 자신과의 유사도) 및 하삼각행렬 제외하고 유사도 값 추출
                # np.triu_indices: 상삼각행렬의 인덱스를 반환 (k=1은 대각선 제외)
                indices = np.triu_indices(num_chunks, k=1)
                unique_similarities = sim_matrix[indices]

                # 전체 유사도 리스트에 추가
                all_similarities.extend(unique_similarities)

                # 기술 통계량 계산 (numpy 사용)
                if len(unique_similarities) > 0:
                    stats = {
                        'note_id': note_id,
                        'num_chunks': num_chunks,
                        'num_pairs': len(unique_similarities),
                        'mean': np.mean(unique_similarities),
                        'std': np.std(unique_similarities),
                        'min': np.min(unique_similarities),
                        '25% (Q1)': np.percentile(unique_similarities, 25),
                        '50% (Median)': np.median(unique_similarities),
                        '75% (Q3)': np.percentile(unique_similarities, 75),
                        'max': np.max(unique_similarities)
                    }
                    results_per_note.append(stats)
                else: # 비교할 쌍이 없는 경우 (이론상 num_chunks >= 2 이므로 발생 안 함)
                     results_per_note.append({'note_id': note_id, 'num_chunks': num_chunks, 'num_pairs': 0})

            except Exception as calc_err:
                print(f"오류: 노트 '{note_id}' 유사도 계산 중 오류 발생: {calc_err}")
        else:
             # 청크가 1개인 노트는 비교 불가
             results_per_note.append({'note_id': note_id, 'num_chunks': num_chunks, 'num_pairs': 0})

    # --- 4. 결과 출력 ---
    print("\n--- 문서별 유사도 통계 결과 ---")
    if results_per_note:
        df_results = pd.DataFrame(results_per_note)
        # 보기 좋게 출력 (note_id 기준 정렬)
        print(df_results.sort_values(by='note_id').to_string())
    else:
        print("분석할 노트가 없습니다.")

    print("\n--- 전체 문서 내 청크 간 유사도 통계 (고유 쌍 기준) ---")
    if all_similarities:
        all_similarities_array = np.array(all_similarities)
        overall_stats = {
            'Total Pairs': len(all_similarities_array),
            'Overall Mean': np.mean(all_similarities_array),
            'Overall Std': np.std(all_similarities_array),
            'Overall Min': np.min(all_similarities_array),
            'Overall 25% (Q1)': np.percentile(all_similarities_array, 25),
            'Overall 50% (Median)': np.median(all_similarities_array),
            'Overall 75% (Q3)': np.percentile(all_similarities_array, 75),
            'Overall Max': np.max(all_similarities_array)
        }
        # 보기 좋게 출력
        for key, value in overall_stats.items():
             # 소수점 4자리까지 표시
             if isinstance(value, (float, np.float32, np.float64)):
                 print(f"{key}: {value:.4f}")
             else:
                 print(f"{key}: {value}")

        # 히스토그램 시각화 (선택 사항, matplotlib 필요: !pip install matplotlib)
        # import matplotlib.pyplot as plt
        # plt.figure(figsize=(10, 6))
        # plt.hist(all_similarities_array, bins=50, color='skyblue', edgecolor='black')
        # plt.title('Distribution of Cosine Similarities between Chunks within Documents')
        # plt.xlabel('Cosine Similarity')
        # plt.ylabel('Frequency')
        # plt.grid(axis='y', alpha=0.75)
        # plt.show()

    else:
        print("계산된 유사도 값이 없습니다.")

    print("\n--- 분석 완료 ---")
    print("위 통계 결과(특히 전체 중앙값, Q3 등)를 참고하여 그룹핑 임계값 후보를 정할 수 있습니다.")
    print("-" * 40)

else:
    print("\n오류: 임베딩 데이터가 로드되지 않아 분석을 시작할 수 없습니다.")

입력 임베딩 파일: /kaggle/working/output/chunk_embeddings.jsonl

--- 임베딩 데이터 로드 및 노트별 그룹핑 시작 ---
총 10개의 노트에서 130개의 청크 임베딩 로드 완료.

--- 각 문서 내 청크 간 유사도 분포 분석 시작 ---


문서별 유사도 분석 중:   0%|          | 0/10 [00:00<?, ?it/s]


--- 문서별 유사도 통계 결과 ---
    note_id  num_chunks  num_pairs      mean       std       min  25% (Q1)  50% (Median)  75% (Q3)       max
0  S0000105          12         66  0.402825  0.222825 -0.080198  0.263595      0.464515  0.574849  0.789231
1  S0000286          12         66  0.506660  0.118531  0.251630  0.429345      0.526255  0.600050  0.784638
2  S0000337          24        276  0.398637  0.125208  0.093367  0.311305      0.405393  0.488760  0.731864
3  S0000473           5         10  0.606027  0.107232  0.455347  0.497419      0.621908  0.694066  0.764980
4  S0000474          15        105  0.501833  0.209834 -0.051862  0.496606      0.568077  0.619850  0.755033
5  S0000565          12         66  0.458255  0.174801  0.024859  0.426608      0.511979  0.567507  0.741638
6  S0000803           5         10  0.372503  0.165366  0.185897  0.216926      0.300597  0.560831  0.599471
7  S0000865          18        153  0.382753  0.179589  0.012226  0.261097      0.404519  0.501824  0.886

In [6]:
# -*- coding: utf-8 -*-
# 필요한 라이브러리 임포트
import os
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import networkx as nx # 그래프 기반 그룹핑을 위해 사용
from tqdm.notebook import tqdm
import pandas as pd # 결과 확인용 (선택 사항)

# --- 1. 설정 (Configuration) ---

INPUT_EMBEDDING_FILE = '/kaggle/working/output/chunk_embeddings.jsonl' # 임베딩 데이터 파일
OUTPUT_GROUP_FILE = '/kaggle/working/output/grouped_chunks_info.jsonl' # 그룹핑 결과 파일
SIMILARITY_THRESHOLD = 0.8  # 유사도 임계값 ### 0.8로 수정됨 ###
MIN_GROUP_SIZE = 2          # 그룹으로 간주할 최소 청크 수

print(f"입력 임베딩 파일: {INPUT_EMBEDDING_FILE}")
print(f"출력 그룹 파일: {OUTPUT_GROUP_FILE}")
print(f"유사도 임계값: {SIMILARITY_THRESHOLD}")
print(f"최소 그룹 크기: {MIN_GROUP_SIZE}")

# --- 2. 데이터 로드 ---

all_chunk_data = [] # 청크 데이터 (메타데이터 + 벡터) 저장 리스트
chunk_embeddings = [] # 벡터만 따로 저장할 리스트 (유사도 계산용)
chunk_lookup = {}     # 인덱스 -> 청크 메타데이터 매핑용 딕셔너리

print(f"\n--- 임베딩 데이터 로딩 시작 ---")
if not os.path.exists(INPUT_EMBEDDING_FILE):
    print(f"오류: 임베딩 파일 '{INPUT_EMBEDDING_FILE}'을 찾을 수 없습니다.")
else:
    try:
        with open(INPUT_EMBEDDING_FILE, 'r', encoding='utf-8') as f_in:
            for i, line in enumerate(f_in):
                try:
                    data = json.loads(line)
                    if 'note_id' in data and 'chunk_id' in data and 'vector' in data and 'text' in data:
                        vector = np.array(data['vector'], dtype=np.float32)
                        if i > 0 and vector.shape[0] != chunk_embeddings[0].shape[0]:
                             print(f"경고: {i}번째 청크 벡터 차원 불일치. 건너<0xEB><0x9B><0x84>니다.")
                             continue
                        all_chunk_data.append(data)
                        chunk_embeddings.append(vector)
                        chunk_lookup[i] = data
                    else: print(f"경고: {i}번째 라인 필수 필드 누락. 건너<0xEB><0x9B><0x84>니다.")
                except json.JSONDecodeError: print(f"경고: 잘못된 JSON 라인 건너뛰기: {line.strip()}")
                except Exception as parse_err: print(f"경고: 데이터 처리 중 오류 ({i}번째 라인): {parse_err}")

        if not all_chunk_data or not chunk_embeddings:
             print("오류: 유효한 임베딩 데이터를 로드하지 못했습니다.")
             chunk_embeddings = None
        else:
            chunk_embeddings = np.array(chunk_embeddings)
            print(f"총 {len(all_chunk_data)}개의 청크 데이터 및 임베딩 로드 완료.")
            print(f"임베딩 배열 형태: {chunk_embeddings.shape}")

    except Exception as e:
        print(f"임베딩 파일 로딩 중 오류 발생: {e}")
        chunk_embeddings = None

# --- 3. 유사도 계산 및 그룹핑 ---

if chunk_embeddings is not None and len(chunk_embeddings) >= MIN_GROUP_SIZE:
    print(f"\n--- 유사도 계산 및 그룹핑 시작 (임계값: {SIMILARITY_THRESHOLD}) ---")

    # 코사인 유사도 계산 (모든 쌍)
    print("코사인 유사도 행렬 계산 중...")
    try:
        similarity_matrix = cosine_similarity(chunk_embeddings)
        print("유사도 행렬 계산 완료.")
    except Exception as sim_err:
        print(f"오류: 유사도 행렬 계산 중 오류 발생: {sim_err}")
        similarity_matrix = None

    if similarity_matrix is not None:
        # 임계값 이상인 쌍들을 엣지로 하는 그래프 생성
        print("유사도 기반 그래프 생성 중...")
        graph = nx.Graph()
        num_chunks = len(all_chunk_data)
        for i in tqdm(range(num_chunks), desc="엣지 추가 중"):
            graph.add_node(i)
            for j in range(i + 1, num_chunks):
                # 부동 소수점 비교 시 작은 오차 고려 (선택적)
                # if similarity_matrix[i, j] >= SIMILARITY_THRESHOLD - 1e-9:
                if similarity_matrix[i, j] >= SIMILARITY_THRESHOLD:
                    graph.add_edge(i, j, weight=similarity_matrix[i, j])

        # 연결된 컴포넌트(그룹) 찾기
        print("연결된 컴포넌트(그룹) 찾는 중...")
        connected_components = list(nx.connected_components(graph))

        # 최소 크기 기준을 만족하는 그룹만 필터링
        valid_groups = [group for group in connected_components if len(group) >= MIN_GROUP_SIZE]
        print(f"총 {len(valid_groups)}개의 유효한 그룹 (크기 >= {MIN_GROUP_SIZE})을 찾았습니다.")

        # --- 4. 그룹 정보 및 액션 대상 생성/저장 ---
        print(f"\n--- 그룹 정보 및 액션 대상 파일 저장 시작 ---")
        grouped_results = []
        group_id_counter = 0

        for group_indices in tqdm(valid_groups, desc="그룹 정보 생성 중"):
            # 그룹 인덱스들을 set으로 변환 (조회 속도 향상)
            group_indices_set = set(group_indices)

            group_data = {
                "group_id": group_id_counter,
                "similarity_threshold": SIMILARITY_THRESHOLD,
                "member_chunks": [],
                "synthesis_input_texts": [],
                "backlink_candidate_notes": set()
            }

            member_texts = []
            note_ids_in_group = set()
            # 그룹 멤버 정보 추가
            for index in group_indices_set:
                chunk_info = chunk_lookup.get(index)
                if chunk_info:
                    member_chunk_info = {
                        "note_id": chunk_info.get("note_id"),
                        "chunk_id": chunk_info.get("chunk_id"),
                        "doc_title": chunk_info.get("doc_title", ""),
                        "filepath": chunk_info.get("filepath", ""),
                        "text_preview": chunk_info.get("text", "")[:100] + "..."
                        # 그룹 내 다른 멤버와의 유사도 정보 추가 (선택적, 계산량 증가)
                        # "similarities_within_group": {}
                    }
                    # # 그룹 내 다른 멤버와의 유사도 계산 (선택적)
                    # for other_index in group_indices_set:
                    #     if index != other_index:
                    #         sim = similarity_matrix[index, other_index]
                    #         member_chunk_info["similarities_within_group"][f"chunk_{other_index}"] = round(sim, 4)

                    group_data["member_chunks"].append(member_chunk_info)
                    member_texts.append(chunk_info.get("text", ""))
                    note_ids_in_group.add(chunk_info.get("note_id"))

            group_data["synthesis_input_texts"] = member_texts
            group_data["backlink_candidate_notes"] = sorted(list(note_ids_in_group))

            # member_chunks 리스트를 chunk_id 기준으로 정렬 (선택적)
            group_data["member_chunks"].sort(key=lambda x: (x["note_id"], x["chunk_id"]))

            grouped_results.append(group_data)
            group_id_counter += 1

        # 최종 결과를 JSON Lines 파일로 저장 (덮어쓰기 'w')
        try:
            with open(OUTPUT_GROUP_FILE, 'w', encoding='utf-8') as f_out:
                for group_result in grouped_results:
                    json_string = json.dumps(group_result, ensure_ascii=False)
                    f_out.write(json_string + '\n')
            print(f"그룹핑 결과가 '{OUTPUT_GROUP_FILE}'에 저장되었습니다.")
        except Exception as write_err:
            print(f"오류: 그룹핑 결과 파일 저장 중 오류 발생: {write_err}")

        print("\n--- 유사 청크 그룹핑 및 액션 정보 생성 완료 ---")
        print("-" * 40)

    else:
         print("오류: 유사도 행렬 계산에 실패하여 그룹핑을 진행할 수 없습니다.")

elif len(chunk_embeddings) < MIN_GROUP_SIZE:
     print(f"\n오류: 로드된 청크 수가 최소 그룹 크기({MIN_GROUP_SIZE})보다 작아 그룹핑을 수행할 수 없습니다.")
else:
    print("\n오류: 임베딩 데이터가 로드되지 않아 그룹핑 프로세스를 시작할 수 없습니다.")

입력 임베딩 파일: /kaggle/working/output/chunk_embeddings.jsonl
출력 그룹 파일: /kaggle/working/output/grouped_chunks_info.jsonl
유사도 임계값: 0.8
최소 그룹 크기: 2

--- 임베딩 데이터 로딩 시작 ---
총 130개의 청크 데이터 및 임베딩 로드 완료.
임베딩 배열 형태: (130, 768)

--- 유사도 계산 및 그룹핑 시작 (임계값: 0.8) ---
코사인 유사도 행렬 계산 중...
유사도 행렬 계산 완료.
유사도 기반 그래프 생성 중...


엣지 추가 중:   0%|          | 0/130 [00:00<?, ?it/s]

연결된 컴포넌트(그룹) 찾는 중...
총 1개의 유효한 그룹 (크기 >= 2)을 찾았습니다.

--- 그룹 정보 및 액션 대상 파일 저장 시작 ---


그룹 정보 생성 중:   0%|          | 0/1 [00:00<?, ?it/s]

그룹핑 결과가 '/kaggle/working/output/grouped_chunks_info.jsonl'에 저장되었습니다.

--- 유사 청크 그룹핑 및 액션 정보 생성 완료 ---
----------------------------------------


In [7]:
# -*- coding: utf-8 -*-
import json
import os

# 그룹핑 결과 파일 경로
GROUP_INFO_FILE = '/kaggle/working/output/grouped_chunks_info.jsonl'

print(f"--- 그룹핑 결과 확인 ({GROUP_INFO_FILE}) ---")

if not os.path.exists(GROUP_INFO_FILE):
    print(f"오류: 그룹핑 결과 파일 '{GROUP_INFO_FILE}'을 찾을 수 없습니다.")
else:
    try:
        with open(GROUP_INFO_FILE, 'r', encoding='utf-8') as f_in:
            group_found = False
            for line in f_in:
                group_found = True
                try:
                    group_data = json.loads(line)
                    group_id = group_data.get("group_id", "N/A")
                    threshold = group_data.get("similarity_threshold", "N/A")
                    member_chunks = group_data.get("member_chunks", [])
                    backlink_notes = group_data.get("backlink_candidate_notes", [])

                    print(f"\n===== 그룹 ID: {group_id} (임계값: {threshold}) =====")
                    print(f"포함된 청크 수: {len(member_chunks)}")
                    print(f"백링크 후보 노트 ID: {backlink_notes}")

                    print("\n--- 포함된 청크 목록 (미리보기) ---")
                    if not member_chunks:
                        print("  (포함된 청크 정보 없음)")
                    else:
                        for i, chunk_info in enumerate(member_chunks):
                            note_id = chunk_info.get("note_id", "?")
                            chunk_id = chunk_info.get("chunk_id", "?")
                            title = chunk_info.get("doc_title", "제목 없음")
                            preview = chunk_info.get("text_preview", "내용 없음")
                            print(f"  {i+1}. [노트:{note_id} / 청크:{chunk_id}] (제목: {title})")
                            print(f"     내용: {preview}")

                    # 통합용 텍스트는 너무 길 수 있으니 필요한 경우 별도 확인
                    # synthesis_texts = group_data.get("synthesis_input_texts", [])
                    # print("\n--- 통합 대상 텍스트 목록 ---")
                    # for i, text in enumerate(synthesis_texts):
                    #     print(f"  {i+1}. {text[:150]}...") # 일부만 출력

                    print("=" * (len(str(group_id)) + 20)) # 구분선

                except json.JSONDecodeError:
                    print(f"\n오류: 잘못된 JSON 라인 발견 - {line.strip()}")
                except Exception as parse_err:
                    print(f"\n오류: 그룹 데이터 처리 중 오류 발생 - {parse_err}")

            if not group_found:
                print("결과 파일은 존재하지만, 유효한 그룹 정보를 찾지 못했습니다.")

    except Exception as e:
        print(f"그룹핑 결과 파일 읽기 중 오류 발생: {e}")

print("\n--- 그룹핑 결과 확인 완료 ---")

--- 그룹핑 결과 확인 (/kaggle/working/output/grouped_chunks_info.jsonl) ---

===== 그룹 ID: 0 (임계값: 0.8) =====
포함된 청크 수: 2
백링크 후보 노트 ID: ['S0000865']

--- 포함된 청크 목록 (미리보기) ---
  1. [노트:S0000865 / 청크:5] (제목: 호흡기감염증 병원체 분포실태 조사를 위한 감시망 구성 및 운영)
     내용: 서울시립 보라매병원, 
3. 인천의료원 
4. 평촌성심병원 
5 춘천 성심병원 
6....
  2. [노트:S0000865 / 청크:12] (제목: 호흡기감염증 병원체 분포실태 조사를 위한 감시망 구성 및 운영)
     내용: 서울시립 보라매병원, 3. 인천의료원 4. 평촌성심병원 5....

--- 그룹핑 결과 확인 완료 ---
